In [246]:
import xarray as xr
import numpy as np
import pandas as pd
import csv
import xesmf as xe
import regionmask

# Constants
rearth = 6.37122e6
area_earth = 4 * np.pi * rearth**2



In [247]:
# Read NetCDF data
def read_netcdf_data(infile):
    with xr.open_dataset(infile) as ds:
        return ds

# Read input data
def read_input_data_raisenan(filename = '../../raw_data_inputs/raisanen_2022_BC_temp/praisanen-Black-carbon-radiative-forcing-and-climate-response-tool-182f7d3/input_data.csv'):
    df = pd.read_csv(filename, index_col = [0])

    emissions = df.loc['Emissions_rate']
    
    if len(emissions) == 1:
        lmonthly_emissions = False
        emissions_annual = emissions[0]
        emissions_monthly = None
    elif len(emissions) == 12:
        lmonthly_emissions = True
        emissions_monthly = xr.DataArray(emissions, dims=['month'], coords={'month': range(1, 13)})
        emissions_annual = None
    else:
        raise ValueError("Invalid number of emission values")
        
    return df.loc['Longitude'][0], df.loc['Longitude'][1], df.loc['Latitude'][0], df.loc['Latitude'][1], lmonthly_emissions, emissions_monthly, emissions_annual

def read_input_data_plants(lat, lon, emissions):
    df = pd.read_csv(filename, index_col = [0])

    emissions = df.loc['Emissions_rate']
    
    if len(emissions) == 1:
        lmonthly_emissions = False
        emissions_annual = emissions[0]
        emissions_monthly = None
    elif len(emissions) == 12:
        lmonthly_emissions = True
        emissions_monthly = xr.DataArray(emissions, dims=['month'], coords={'month': range(1, 13)})
        emissions_annual = None
    else:
        raise ValueError("Invalid number of emission values")
        
    return lat, lon, emissions, lmonthly_emissions, emissions_monthly, emissions_annual


# Helper functions
def modify_and_check_lon(rlon1, rlon2):
    if rlon2 < rlon1:
        if rlon2 < 0:
            rlon2 += 360
        elif rlon1 > 0:
            rlon1 -= 360
    
    ok = True
    if rlon2 - rlon1 < 0 or rlon2 - rlon1 > 360 or rlon1 < -360 or rlon2 > 360:
        ok = False
    
    return rlon1, rlon2, ok

def check_lat(rlat1, rlat2):
    return -90 <= rlat1 <= rlat2 <= 90

def global_annual_mean_rf(rf, overlap_area, sec_per_season, sec_per_year):
    # Calculate the weighted sum across seasons
    seasonal_sum = (rf*overlap_area*sec_per_season).sel(season=['DJF', 'MAM', 'JJA', 'SON']).sum(dim=['lat', 'lon', 'season'])
    annual_sum = (rf*overlap_area*sec_per_season).sel(season = 'ANN').sum(dim = ['lat','lon'])

    TJ_to_J  = 1e12
    print(f"RF seasonal sum: {seasonal_sum.sum().item():.5e}")
    print(f"Total overlap area: {overlap_area.sum().values:.5e}")
    print(f"Total seconds per year: {sec_per_year:.5e}")
    
    seasonal_mean = seasonal_sum *TJ_to_J / (area_earth * sec_per_year)
    annual_mean = annual_sum *TJ_to_J / (area_earth * sec_per_year)
    return (seasonal_mean, annual_mean)

def global_annual_mean_dt(dt, overlap_area, sec_per_season, sec_per_year):
    # Calculate the weighted sum across seasons
    seasonal_sum = (overlap_area * dt * sec_per_season).sel(season=['DJF', 'MAM', 'JJA', 'SON']).sum(dim=['lat', 'lon', 'season'])
    annual_sum = (overlap_area * dt * sec_per_season).sel(season = 'ANN').sum(dim = ['lat','lon'])

    TJ_to_J  = 1e12

    seasonal_mean = seasonal_sum / (sec_per_year)
    annual_mean = annual_sum / (sec_per_year)

    return (seasonal_mean, annual_mean)
def regrid(ds, lon_w, lon_e, lat_s, lat_n, target_resolution=0.5):

    ds_out = xr.Dataset(
    {
        "lat": (["lat"], np.arange(-90, 90, target_resolution), {"units": "degrees_north"}),
        "lon": (["lon"], np.arange(-180, 180, target_resolution), {"units": "degrees_east"}),
    }
)

    # Create regridders
    regridder_ds = xe.Regridder(ds, ds_out, 'conservative_normed')

    ds_reg = regridder_ds(ds)

    
    return ds_out, ds_reg

def mask(ds_reg, lon_w, lon_e, lat_s, lat_n, target_resolution=1):
    
    # Create a mask for the region
    rg = np.array([[lon_w, lat_s], [lon_e, lat_s], [lon_e, lat_n], [lon_w, lat_n]])
    region = regionmask.Regions([rg], names=['emissions_loc'])
    mask_frac = region.mask_3D_frac_approx(ds_reg)

    ds_masked = ds_reg*mask_frac
    
    return ds_masked, mask_frac



def find_area(ds_masked, R=6378137):
    """ 
    Calculate the area of each grid cell from lat/lon coordinates.
    
    Parameters:
    ds (xarray.Dataset): Dataset containing 'lat' and 'lon' coordinates
    R (float): Earth's radius in km (default: 6378.1 km)
    
    Returns:
    xarray.DataArray: Grid cell areas in square km
    """
    # Ensure latitudes and longitudes are in ascending order
    ds_masked = ds_masked.sortby('lat').sortby('lon')

    # Convert to radians
    lat = np.deg2rad(ds_masked['lat'])
    lon = np.deg2rad(ds_masked['lon'])

    # Calculate differences
    dlat = lat.diff(dim='lat')
    dlon = lon.diff(dim='lon')

    # Calculate the area using the trapezoidal formula
    dx1 = R * dlon * np.cos(lat)
    dx2 = R * dlon * np.cos(lat[1:] + dlat)
    dy = R * dlat

    A = 0.5 * (dx1 + dx2) * dy
    
    return A.transpose()



In [248]:
def main():
    
    infile = "../../raw_data_inputs/raisanen_2022_BC_temp/praisanen-Black-carbon-radiative-forcing-and-climate-response-tool-182f7d3/BC_forcing_and_climate_response_normalized_by_emissions.nc"
    ds = read_netcdf_data(infile)
    #ds = xr.ones_like(ds)
    ds = ds.rename({'nseason': 'season'})


    # Define seasons
    seasons = ['ANN', 'DJF', 'MAM', 'JJA', 'SON']

    # Read input data
    lon_w, lon_e, lat_s, lat_n, lmonthly_emissions, emissions_monthly, emissions_annual = read_input_data()

    ds = ds.assign_coords({'season':seasons})


    # Check and modify longitude and latitude
    lon_w, lon_e, ok = modify_and_check_lon(lon_w, lon_e)
    if not ok:
        raise ValueError(f"Longitude limits not OK: {lon_w}, {lon_e}")

    if not check_lat(lat_s, lat_n):
        raise ValueError(f"Latitude limits not OK: {lat_s}, {lat_n}")

    # Calculate seasonal emissions
    ndays_per_month = xr.DataArray([31, 28, 31, 30, 31, 30, 31, 31, 30, 31, 30, 31], dims=['month'])
    sec_per_month = 86400 * ndays_per_month
    months_in_season = xr.DataArray([
        [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
        [1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1],
        [0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0]
    ], dims=['season', 'month'], coords = {'season':seasons})


    sec_per_season = (months_in_season * sec_per_month).sum('month')
    sec_per_year = sec_per_season.sel(season='ANN')



    # Calculate seasonal emissions
    if lmonthly_emissions:
        emissions_seasonal = (months_in_season * sec_per_month * emissions_monthly).sum(dim='month') / sec_per_season
    else:
        emissions_seasonal = xr.DataArray(np.zeros(len(seasons)), dims=['season'], coords={'season': seasons})
        emissions_seasonal.loc[{'season': 'ANN'}] = emissions_annual


    # Regrid and mask the dataset
    ds_out, ds_reg = regrid(ds, lon_w, lon_e, lat_s, lat_n)
    ds_masked_raw, mask_frac = mask(ds_reg, lon_w, lon_e, lat_s, lat_n)
    ds_masked = ds_masked_raw * emissions_seasonal


    # Calculate the overlap area
    cell_area = find_area(ds_masked)
    overlap_area = cell_area * mask_frac
    
    ##checks
    total_area = cell_area.sum().values
    
    def calculate_expected_area(lat_s, lat_n, lon_w, lon_e, R=6371000):
        """
        Calculate the expected area of a region on Earth's surface.

        Parameters:
        lat_s, lat_n: South and North latitudes in degrees
        lon_w, lon_e: West and East longitudes in degrees
        R: Earth's radius in meters (default: 6,371,000 m)

        Returns:
        Area in square meters
        """
        lat_s, lat_n = np.radians(lat_s), np.radians(lat_n)
        lon_w, lon_e = np.radians(lon_w), np.radians(lon_e)

        return R**2 * abs(np.sin(lat_n) - np.sin(lat_s)) * abs(lon_e - lon_w)
    expected_area = calculate_expected_area(lat_s, lat_n, lon_w, lon_e)

    # Calculate global annual mean radiative forcings and temperature responses
    drf_toa_out_ssn, drf_toa_out_ann = global_annual_mean_rf(ds_masked.dirsf_toa, overlap_area, sec_per_season, sec_per_year)
    snowrf_toa_out_ssn, snowrf_toa_out_ann = global_annual_mean_rf(ds_masked.snowsf_toa, overlap_area, sec_per_season, sec_per_year)
    dt_drf_out_ssn, dt_drf_out_ann = global_annual_mean_dt(ds_masked.dt_norm_drf, overlap_area, sec_per_season, sec_per_year)
    dt_snowrf_out_ssn, dt_snowrf_out_ann = global_annual_mean_dt(ds_masked.dt_norm_snowrf, overlap_area, sec_per_season, sec_per_year)

    # Print comparison with expected values
    expected_drf_toa = 1.16930e-04
    expected_snowrf_toa = 9.19170e-05
    expected_dt_drf = 1.14593e-04
    expected_dt_snowrf = 1.94513e-04

  
    return(drf_toa_out_ssn, drf_toa_out_ann, snowrf_toa_out_ssn, snowrf_toa_out_ann, dt_drf_out_ssn, dt_drf_out_ann, dt_snowrf_out_ssn, dt_snowrf_out_ann , ds_masked)

In [249]:
if __name__ == "__main__":
    drf_toa_out_ssn, drf_toa_out_ann, snowrf_toa_out_ssn, snowrf_toa_out_ann, dt_drf_out_ssn, dt_drf_out_ann, dt_snowrf_out_ssn, dt_snowrf_out_ann , ds_masked = main()

/tmp/ipykernel_540/1001531317.py:23: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  return df.loc['Longitude'][0], df.loc['Longitude'][1], df.loc['Latitude'][0], df.loc['Latitude'][1], lmonthly_emissions, emissions_monthly, emissions_annual
/home/emfreese/miniconda3/envs/bc-irf/lib/python3.9/site-packages/xesmf/backend.py:56: UserWarning: Latitude is outside of [-90, 90]
  warnings.warn('Latitude is outside of [-90, 90]')
/home/emfreese/miniconda3/envs/bc-irf/lib/python3.9/site-packages/xesmf/backend.py:56: UserWarning: Latitude is outside of [-90, 90]
  warnings.warn('Latitude is outside of [-90, 90]')


RF seasonal sum: 1.77611e+06
Total overlap area: 4.14510e+11
Total seconds per year: 3.15360e+07
RF seasonal sum: 1.30610e+06
Total overlap area: 4.14510e+11
Total seconds per year: 3.15360e+07


In [250]:
print("\nComparison with expected values:")

# DRF_TOA
print("DRF_TOA:")
print(f"  Seasonal: Calculated = {drf_toa_out_ssn.item():.5e}, Ratio to Expected = {(drf_toa_out_ssn / expected_drf_toa).item():.5f}")
print(f"  Annual:   Calculated = {drf_toa_out_ann.item():.5e}, Ratio to Expected = {(drf_toa_out_ann / expected_drf_toa).item():.5f}")
print(f"  Expected = {expected_drf_toa:.5e}")

# SNOWRF_TOA
print("\nSNOWRF_TOA:")
print(f"  Seasonal: Calculated = {snowrf_toa_out_ssn.item():.5e}, Ratio to Expected = {(snowrf_toa_out_ssn / expected_snowrf_toa).item():.5f}")
print(f"  Annual:   Calculated = {snowrf_toa_out_ann.item():.5e}, Ratio to Expected = {(snowrf_toa_out_ann / expected_snowrf_toa).item():.5f}")
print(f"  Expected = {expected_snowrf_toa:.5e}")

# DT_DRF
print("\nDT_DRF:")
print(f"  Seasonal: Calculated = {dt_drf_out_ssn.item():.5e}, Ratio to Expected = {(dt_drf_out_ssn / expected_dt_drf).item():.5f}")
print(f"  Annual:   Calculated = {dt_drf_out_ann.item():.5e}, Ratio to Expected = {(dt_drf_out_ann / expected_dt_drf).item():.5f}")
print(f"  Expected = {expected_dt_drf:.5e}")

# DT_SNOWRF
print("\nDT_SNOWRF:")
print(f"  Seasonal: Calculated = {dt_snowrf_out_ssn.item():.5e}, Ratio to Expected = {(dt_snowrf_out_ssn / expected_dt_snowrf).item():.5f}")
print(f"  Annual:   Calculated = {dt_snowrf_out_ann.item():.5e}, Ratio to Expected = {(dt_snowrf_out_ann / expected_dt_snowrf).item():.5f}")
print(f"  Expected = {expected_dt_snowrf:.5e}")



Comparison with expected values:
DRF_TOA:
  Seasonal: Calculated = 1.10410e-04, Ratio to Expected = 0.94424
  Annual:   Calculated = 1.24629e-04, Ratio to Expected = 1.06584
  Expected = 1.16930e-04

SNOWRF_TOA:
  Seasonal: Calculated = 8.11922e-05, Ratio to Expected = 0.88332
  Annual:   Calculated = 7.05777e-05, Ratio to Expected = 0.76784
  Expected = 9.19170e-05

DT_DRF:
  Seasonal: Calculated = 1.05981e-04, Ratio to Expected = 0.92484
  Annual:   Calculated = 1.20226e-04, Ratio to Expected = 1.04916
  Expected = 1.14593e-04

DT_SNOWRF:
  Seasonal: Calculated = 1.71504e-04, Ratio to Expected = 0.88171
  Annual:   Calculated = 1.49556e-04, Ratio to Expected = 0.76887
  Expected = 1.94513e-04
